In [ ]:
import pandas as pd
import spacy
from nltk.corpus import wordnet as wn

In [3]:
nlp = spacy.load("en_core_web_sm")

In [33]:
data = pd.read_csv('data/it-corpus.tsv', sep='\t')

In [34]:
data

,Class,Sentence
0,NomAnaph,"Nevertheless, in view of the world-wide ..."
1,ClauseAnaph,It's simply bare-faced fortune hunting; b...
2,NomAnaph,Mary was situated about two miles from ...
3,ClauseAnaph,She is up at five every morning to mil...
4,ClauseAnaph,Don't say it.
...,...,...
508,NomAnaph,I was fairly certain that it was Mr.s. ...
509,ClauseAnaph,It all arose from a misunderstanding.
510,ClauseAnaph,He knew it.
511,NomAnaph,"Pardon me, mon ami, but you did not un..."


In [35]:
rows = []

In [4]:
description = spacy.explain("pobj")
print(description)

object of preposition


In [ ]:
def process_row(row):
    sentence = row.Sentence
    doc = nlp(sentence)

    # obtain f1
    f_one = -1
    for i, token in enumerate(doc):
        if token.text.lower() == 'it' and token.pos_ == "PRON":
            f_one = i + 1
            break
    
    it_index = f_one - 1
    
    # obtain f2
    f_two = len(doc)
    
    #obtain f3
    f_three = 0
    for i, token in enumerate(doc):
        if token.is_punct:
            f_three+=1
            
    #obtain f4
    f_four = 0
    for np in doc.noun_chunks:
        if np.end <= it_index:
            f_four +=1
            
    #obtain f5
    f_five = 0
    for np in doc.noun_chunks:
        if np.start > it_index:
            f_five += 1
            
    #obtain f6
    f_six = False
    for tok in doc:
        if tok.pos_ == "ADP":  
            end_i = max(t.i for t in tok.subtree)
            if end_i == it_index - 1:
                f_six = True
    
    #obtain f7
    f_seven = ['ABS','ABS','ABS','ABS','ABS','ABS','ABS','ABS']    
    preceding = doc[max(0, it_index-4):it_index]
    for i, token in enumerate(preceding):
        f_seven[4 - len(preceding) + i] = token.pos_
        
    succeding = doc[it_index + 1:it_index + 5]
    for i, token in enumerate(succeding):
        f_seven[4 + i] = token.pos_
    
    #obtain f8
    f_eight = False
    if f_one < len(doc):
        if doc[f_one].text.lower().endswith('ing') and doc[f_one].pos_ == 'VERB':
            f_eight = True
    
    #obtain f9
    f_nine = False
    if f_one < len(doc):
        if doc[f_one].pos_ == "ADP":
            f_nine = True
    
    #obtain f10
    f_ten = 0
    for i in range(it_index+1, len(doc)):
        if doc[i].pos_ == "ADJ":
            f_ten += 1
            
    #obtain f11
    f_eleven = False
    if it_index-1 >= 0:
        if doc[it_index - 1].pos_ == "VERB":
            f_eleven = True
    
    #obtain f12
    f_twelve = False
    if it_index + 1 < len(doc):
        if doc[it_index + 1].pos_ == "VERB":
            f_twelve = True

    #obtain f13
    f_thirteen = False
    if it_index + 1 < len(doc):
        if doc[it_index + 1].pos_ == "ADJ":
            f_thirteen = True
    
    #obtain f14
    f_fourteen = False
    for np in doc.noun_chunks:
        if np.start > it_index:
            for t in np:
                if t.pos_ == "ADJ":
                    f_fourteen = True
                    break
    
    #obtain f15
    f_fifteen = 0
    for i in range(len(doc) - 1):
        if doc[i].lemma_ == "to" and doc[i].pos_ in {"PART", "ADP"} and doc[i+1].pos_ == "VERB":
            f_fifteen = i + 1
            break
    
    #obtain 16
    f_sixteen = 0
    for j in range(it_index + 1, len(doc)):
        if doc[j].pos_ == "ADP":
            f_sixteen = j - it_index - 1
            break

    
    #obtain f17
    f_seventeen = False
    np_starts = []
    for np in doc.noun_chunks:
        if np.start > it_index:
            np_starts.append(np.start)
        
    j = it_index + 1
    while j < len(doc) - 1:
        if doc[j].pos_ == "ADJ":
            if (j + 1) in np_starts:
                f_seventeen = True
                break
        j += 1

    #obtain f18
    f_eighteen = doc[it_index].dep_

    #obtain f19
    f_nineteen = False
    if it_index+1 < len(doc) and doc[it_index+1].pos_ == "VERB":
        synsets = wn.synsets(doc[it_index+1].lemma_, pos=wn.VERB)
        f_nineteen = any(s.lexname() == 'verb.weather' for s in synsets)

    #obtain f20
    f_twenty = False
    if it_index+1 < len(doc) and doc[it_index+1].pos_ == "VERB":
        synsets = wn.synsets(doc[it_index+1].lemma_, pos=wn.VERB)
        f_twenty = any(s.lexname() == 'verb.cognition' for s in synsets)
        
    return {
        "F1": f_one, "F2": f_two, "F3": f_three, "F4": f_four, "F5": f_five,
        "F6": f_six, "F7": f_seven, "F8": f_eight, "F9": f_nine, "F10": f_ten,
        "F11": f_eleven, "F12": f_twelve, "F13": f_thirteen, "F14": f_fourteen, "F15": f_fifteen,
        "F16": f_sixteen, "F17": f_seventeen, "F18": f_eighteen, "F19": f_nineteen, "F20": f_twenty
    }

In [51]:
for row in data.itertuples():
    # print(row)
    k = process_row(row)
    if k is not None:
        print(k)

Nevertheless
It
Mary
She
Do
In
The
It
It
And
But
Do
Might
Associate
It
She
Well
I
I
Yes
We
Cynthia
So
Well
The
His
Mary
Then
I
It
I
I
He
He
He
It
It
The
The
The
We
Wilkins
I
And
He
She
Therefore
In
He
He
He
He
He
He
It
However
He
A
Poirot
Mon
Mon
Mon
It
It
But
He
Finally
No
It
It
It
I
The
I
We
It
I
I
I
I
She
What
Well
She
She
She
She
She
That
I
Ah
poirot
Five
I
A
It
But
I
It
Yes
Yes
What
Who
Did
No
Then
Then
Then
The
And
What
And
Annie
What
Seeing
Coarse
I
I
I
I
So
So
I
Yes
It
I
I
She
Yes
That
It
It
It
It
He
It
A
The
Yes
But
I
Is
I
I
We
I
I
If
If
If
The
Surely
It
It
You
No
It
It
Can
I
It
It
Where
He
He
In
If
Dorcas
Manning
He
She
What
Well
Well
Mr.
John
How
Now
Find
Do
Do
Taking
I
But
But
As
Probably
It
It
Finding
I
And
It
It
It
I
What
It
It
It
Did
And
To
I
It
You
That
Now
Now
Every
She
For
It
It
Shorn
The
It
No
It
cocoa
Dr
Sounded
My
Strychnine
Dorcas
Dorcas
The
Poirot
It
I
I
I
It
Will
He
It
He
He
It
But
You
But
The
I
But
When
But
I
It
I
It
No
Because
He
Would
And
It
Oh
Did
No
The
But